In [16]:
#import semua library yang dibutuhkan
import pandas as pd
import requests
import bs4
import time
from fake_useragent import UserAgent
import math
import re
import random
from sklearn.ensemble import RandomForestClassifier

In [17]:
#membaca data training
df = pd.read_csv("isaPerson.csv")

In [18]:
#membaca data testinf
td = pd.read_csv('tdsd.tsv', sep="\t")

In [19]:
#membuat dataframe baru untuk menampung hasil
dfresult = {'0':'', '1':''}
nc = pd.DataFrame(index=td.index.copy(), columns=dfresult)

In [20]:
df.head()

,contributors_enabled,created_at,default_profile,default_profile_image,description,entities,favourites_count,followers_count,friends_count,geo_enabled,...,profile_use_background_image,protected,screen_name,statuses_count,url,verified,username,isaPerson,userLocation,jobArea
0,False,08/02/2015 0:32,True,False,NaN,NaN,1,16858,6,False,...,True,False,otodriver,17695,NaN,False,@otodriver,nonperson,jabodetabek,hobi
1,False,07/07/2009 13:13,False,False,Markas Pusat Palang Merah Indonesia | Jalan Je...,NaN,2247,1033888,1235,True,...,True,False,palangmerah,53538,https://t.co/mx3e3UYXPi,True,@palangmerah,nonperson,jabodetabek,kesehatan
2,False,16/11/2011 10:36,False,False,Ada diskon setiap hari cek di aplikasi #Pegipe...,NaN,1439,46217,36,True,...,True,False,pegipegi,35329,https://t.co/lP16dITceB,True,@pegipegi,nonperson,jabodetabek,hospitality & tourism
3,False,13/04/2008 03:30,False,False,#Ngahiji Menang Bersama! Download #PERSIBApp ð...,NaN,1464,3502961,90,True,...,True,False,persib,46556,https://t.co/hqHLwIrxfY,True,@persib,nonperson,jawa barat dan banten,olahraga
4,False,19/09/2014 03:05,False,False,Akun Resmi | Dikelola Subbaghumas | Call Cente...,NaN,316,26707,204,True,...,False,False,polresjogja,9204,https://t.co/XrZB8YcypN,True,@polresjogja,nonperson,jawa tengah dan yogyakarta,sosial kemasyarakatan


In [21]:
#memilih kolom-kolom yang mungkin membantu prediksi

used_column = ['default_profile', 'default_profile_image', 'favourites_count', 'followers_count', 'friends_count', 'geo_enabled',
       'protected', 'statuses_count', 'verified', 'isaPerson', 'jobArea']

### Features Engineering

In [22]:
#mengisi semua value kosong pada data testing dengan nilai 0

td = td.fillna(0)

In [23]:
#mentrasformasi value string dan boolean menjadi value numerik

cleanup = {"geo_enabled":{True: 2, False: 1, "": 0},
           "isaPerson":{"person": 1, "nonperson": 0},
           "protected":{True: 2, False: 1, "": 0},
           "verified":{True: 2, False: 1, "": 0},
           "default_profile_image":{True: 2, False: 1, "": 0},
           "jobArea":{"pendidikan dan penelitian": 0, "sains dan teknologi": 1, "kesehatan": 2, "ekonomi dan bisnis": 3,
                     "sosial kemasyarakatan": 4, "media": 5, "hospitality & tourism": 6, "olahraga":7, "hiburan": 8,
                     "seni": 9, "hobi": 10}
          }
df = df.replace(cleanup)
td = td.replace(cleanup)

In [24]:
#mengubah semua tipe data menjadi numerik

for i in used_column:
    df[i] = df[i].astype(int)
    td[i] = td[i].astype(int)

In [25]:
#melihat korelasi dari tiap atribut

df.corr()

,contributors_enabled,default_profile,default_profile_image,entities,favourites_count,followers_count,friends_count,geo_enabled,notifications,profile_use_background_image,protected,statuses_count,verified,isaPerson,jobArea
contributors_enabled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
default_profile,NaN,1.000000,0.068626,NaN,0.023548,-0.099084,-0.029246,-0.275073,NaN,0.331187,0.063962,-0.095064,-0.094704,0.068538,-0.025257
default_profile_image,NaN,0.068626,1.000000,NaN,-0.010739,-0.011508,-0.006521,-0.058317,NaN,0.022728,0.138548,-0.010381,-0.026623,0.066323,0.054132
entities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
favourites_count,NaN,0.023548,-0.010739,NaN,1.000000,0.024048,0.037130,0.077800,NaN,-0.006671,-0.036206,0.031956,0.056712,0.132259,-0.042897
followers_count,NaN,-0.099084,-0.011508,NaN,0.024048,1.000000,0.064445,0.080927,NaN,0.025047,-0.031221,0.243193,0.274662,0.114435,0.075048
friends_count,NaN,-0.029246,-0.006521,NaN,0.037130,0.064445,1.000000,0.043541,NaN,0.031952,-0.024649,0.073574,0.002004,0.004499,0.015296
geo_enabled,NaN,-0.275073,-0.058317,NaN,0.077800,0.080927,0.043541,1.000000,NaN,-0.070299,-0.092130,0.106833,0.186952,-0.043265,-0.025116
notifications,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
profile_use_background_image,NaN,0.331187,0.022728,NaN,-0.006671,0.025047,0.031952,-0.070299,NaN,1.000000,0.079100,-0.006513,-0.089465,0.132195,0.104366


In [26]:
td.dtypes

contributors_enabled             object
created_at                       object
default_profile                   int32
default_profile_image             int32
description                      object
entities                        float64
favourites_count                  int32
followers_count                   int32
friends_count                     int32
geo_enabled                       int32
id_str                           object
name                             object
notifications                    object
profile_background_color         object
profile_link_color               object
profile_location                 object
profile_sidebar_border_color     object
profile_sidebar_fill_color       object
profile_text_color               object
profile_use_background_image     object
protected                         int32
screen_name                      object
statuses_count                    int32
url                              object
utc_offset                      float64


In [27]:
#memilih fitur-fitur untuk penerapan machine learning

cba = ['default_profile_image', 'followers_count', 'friends_count', 'protected', 'profile_use_background_image']

train_X = df[cba]
train_y = df.jobArea

In [28]:
#inisiasi model machine learning random forest

gbc = RandomForestClassifier(random_state=1)

In [29]:
gbc.fit(train_X, train_y)
pred = gbc.predict(td[cba])
nc['1'] = pred

In [203]:
nc.head()

,0,1
0,NaN,4
1,NaN,8
2,NaN,5
3,NaN,5
4,NaN,8


In [30]:
#melakukan transformasi dari value numerik menjadi value string kembali

for i in range(4153):
    row = nc.loc[i, '1']
    nc.loc[i, '0'] = i
    if(row == 0):
        nc.loc[i, '1'] = "pendidikan dan penelitian"
    elif(row == 1):
        nc.loc[i, '1'] = "sains dan teknologi"
    elif(row == 2):
        nc.loc[i, '1'] = "kesehatan"
    elif(row == 3):
        nc.loc[i, '1'] = "ekonomi dan bisnis"
    elif(row == 4):
        nc.loc[i, '1'] = "sosial kemasyarakatan"
    elif(row == 5):
        nc.loc[i, '1'] = "media"
    elif(row == 6):
        nc.loc[i, '1'] = "hospitality & tourism"
    elif(row == 7):
        nc.loc[i, '1'] = "olahraga"
    elif(row == 8):
        nc.loc[i, '1'] = "hiburan"
    elif(row == 9):
        nc.loc[i, '1'] = "seni"
    elif(row == 10):
        nc.loc[i, '1'] = "hobi"

In [31]:
nc.head()

,0,1
0,0,sosial kemasyarakatan
1,1,hiburan
2,2,media
3,3,media
4,4,hiburan


In [206]:
td['jobArea'] = td['jobArea'].fillna('')

In [207]:
td['jobArea'].unique()

array([0])

In [208]:
td = pd.read_csv('tdsd.tsv', sep="\t")

In [209]:
td['description'] = td['description'].fillna('')
td['name'] = td['name'].fillna('')

### Scrapping

In [210]:
import requests
import bs4
from random import randint

selected_sd = {'username':'', 'name':'', 'description': ''}

collection = pd.DataFrame(index=td.index.copy(), columns=selected_sd)

for i in range(0,4153):
    desc = ""
    name = ""
    if(td.loc[i, 'description'] != ''):
        desc = td.loc[i, "description"].lower()
    if(td.loc[i, 'name'] != ''):
        desc += " "
        name = td.loc[i, "name"].lower()
        name = re.sub(r'[^\w]', ' ', name)
        name = re.sub(r'[!@#$]', ' ', name)
        name = td.loc[i, "name"].lower()
        
        #menghapus semua character pada teks
        name = name.replace(",", ' ')
        name = name.replace(".", ' ')
        name = name.replace("/", ' ')
        name = name.replace("|", ' ')
        name = name.replace("&", ' ')
        name = name.replace("_", ' ')
        name = name.replace("(", ' ')
        name = name.replace(")", ' ')
        desc += name
    selected_sd['username'] = td.loc[i, "username"]
    selected_sd['description'] = re.sub(r'[^\w]', ' ', desc)
    selected_sd['description'] = re.sub(r'[!@#$]', ' ', desc)
    
    #menghapus semua character pada teks
    desc = desc.replace(",", ' ')
    desc = desc.replace(".", ' ')
    desc = desc.replace("/", ' ')
    desc = desc.replace("|", ' ')
    desc = desc.replace("&", ' ')
    desc = desc.replace("_", ' ')
    desc = desc.replace("(", ' ')
    desc = desc.replace(")", ' ')
    selected_sd['description'] = desc.replace(",", ' ')
    print(selected_sd['description'])
    print("=========")
    collection.loc[i] = pd.Series(selected_sd)

industrial engineering of pancasila university   industrial management of bogor agricultural university   wilmar chemical indonesia   #viscabarã§a bobid adams
official account of grand indonesia a one-stop shopping for everyone  ph: +62  21  23580001 2 grand indonesia
official account of persis solo  #laskarsambernyawa persis solo
i am you hammam silmi
 chris ryantonius
@ponyourtone â€” booking inquiries : +62 878-8216-7715 dipha barus
manusia biasa yang sedang bertumbuh dan berproses  karin novilda
fasilkom ui 2014 clarita nainggolan
official account institut teknologi sepuluh nopember  mohon support dan bantuannya akun sedang dalam pemulihan sobat kampus!  institut teknologi sepuluh nopember
be grateful  sasono
ctupa7@gmail com carlo tupa indriauan
 hen
do what u like fina finanda
reverse psychology is the ultimate sarcasm wanofas 2
konsultan remaja terbaik!!!! k' rio
akun resmi dpd partai demokrat daerah istimewa yogyakarta ðÿ‡®ðÿ‡© demokrat diy
kalau hidup penuh pilihan  sekarang s

ceo of not giving a fuck pai
ipdn xxviii ario
official twitter dari dapur kobe  dapatkan info menarik seputar resep dan masakan disini https:  t co xemgrq80ov dapur kobe
don't focus on what u lost focus on what u have christopherrusdianto
hiðÿ‘‹ lebih aktif di ig @citrayu citra ayu pranajaya wibrado
kalau ada orang lain kenapa harus saya? http:  t co lh7g85xm2f http:  t co kqw7fv1nuw http:  t co xk8fwsgep9 ary ramadhan
 yuniar dinda
computer science graduate turn into product manager and brand manager   loves to talk about product design business politic and human m  firza pratama
find though she be but little she is fierce actress currently living in jakarta  one   only new twitter instagram: @ irishbella  irish bella
software engineer #womentechmakers @wtmsemarang indonesia farahluthfioktarina
actor announcer presenter recently host #katakanputus transtv senin-jumat 14 30wib  management @mahakaryainc komo ricky
back  well at least   hapis
lagi doyan makan sauzan
cs ui 16 bintang ilha

indonesian actor armando jordy
tempat magang hampir semua tech youtubers indonesia angkatan awal  contact : info@gadgetgaul com ig : gadgetgaul
karya dan kekaryaan ali ghiffar putra rinanto
 andhika pratama
cp : @meialwie  meirina  0812 81022637 c
 abi tuasikal#12
ikatan mahasiswa teknik industri ftui 2018 #ingenious cp: erwin  081380895149    putri  085695881887    rayi  08990043674  imti ftui
berbagi potret dan cerita  menziarahi dan mengobrolkan masa lalu mengingat kini  ðÿ“§ potretlawas@outlook com potret lawas
akun resmi balai produksi dan informasi av - pusdatin kementerian pupr   surabaya indonesia   email : produksiav@gmail com bpiav pusdatin pupr
 imroatul maghfiroh
 ishadi sk
situs jual beli otomotif online terbesar di indonesia  olx co id
terms and conditions apply  #beko armin sunardi
 jasmina vidi
cp: ericsteven +628176824050tt #terburucinta https:  t co xfjpylqiic and join my fanpage on facebook angel pieters
#savethechildren #ngo #bostonchildren'shospital ù†ùûœø²ùžø§niz

wakil ketua badan pengkajian mpr ri â€¢ anggota komisi xi dpr ri â€¢ wakil ketua umum depinas soksi agun gunandjar s
keep staying on the right side #autofollback allfaithisðÿ‡®ðÿ‡©ðÿ†”
psychology   understanding of human omgendut
the leading fixed telecom operator in indonesia provides network internet data center cloud computing and pay tv services  call biznet: 1500988 biznet networks
sometimes i sing stuffs on smule and youtube https:  t co ebkdi5lr53   tweet when bored  feel free to poke! id   en ok!   enfp   dp by @yuu rihime  renma
teknik informatika baharkhan
pingin keliling indonesia kristian patrasio pasaribu
 p e r t i w i
this is me!!! 37 andakara - ftui 2016 naufal farras ahadan
situs untuk pecinta buku ðÿ“– dapatkan buku favoritmu di sini ðÿ‘‡ðÿ» gramedia com
twitter = apa yang sedang terjadi  bantuan: https:  t co heevrrl4yn  twitter indonesia
85#32  34 2014  fasilkom ui lia sumarliyanti
u know who i am kamil
spensa65  bunga azerlina
dust and shadow kelly marvella å¼ å˜‰

panjang umur upaya-upaya baik m alfatih timur
i hate cockroaches  business and cp: @barlieve or email to barlie ve@gmail com ðÿ’‹ wa: 085921315769 adellina fitriyani
penikmat nasi uduk semur jengkol akhmad afrizal
fdlhns username :
 ditya
 debby anita
sahabat keluarga indonesia   official account   facebook: bankbtn   instagram: bankbtn   youtube: bank btn   contact center 1500286 bank btn
saatnya #buatkeputusancerdas bersama intel intel indonesia
flutter community in jakarta indonesia  flutter jakarta
akun resmi divisi humas polri  ig: divisihumaspolri - fb: divisi humas polri - yt: divisi humas polri obyektif dipercaya partisipasi  https:  t co rj40czqmft divisi humas polri
 rania
 anisa ainur rohma
official account of pt  angkasa pura ii  persero  smart connected airport in the region  contact us : https:  t co ik4fa94aww angkasa pura ii
tujuh satu spensa alfrizki widika
website resmi pemerintah provinsi sumatera barat alamat : jalan jenderal sudirman no 51 jati baru padang timur pa

...

 adrian hartanto
actor rangga azof
fakultas ekonomi prodi manajemen unsri 2014 rahmat dwi kurniawan
fisip untan pontianak 2008   @ici pontianak curva nord 69 muhammad nur kahfi
akun resmi dewan perwakilan mahasiswa fakultas matematika dan ilmu pengetahuan alam universitas islam indonesia  dpm fmipa uii  dpm fmipa uii
akun resmi programa 1 radio republik indonesia ternate   pusat pemberdayaan masyarakat   fm 101 8 mw 891 rri play app   https:  t co fhsju0jvxu fm 101 8 pro1 ternate
aeon was first establish in japan and become the largest retailer in asia  aeon mall bsd city is the first aeon mall in indonesia  aeon mall bsd city
ordinary teenager with all the hormones and angst   ragasatya   gvo'14   ilmu komputer ui '14 sang agung raditya
the official twitter account of mcdonald's indonesia mcdonald's indonesia
 punten
basketball   management sampoerna university'14   proverbs 23:18 billy kristanda
farmasi uho 2010    pspa unud xii    iqo art management    ig : agmapriliyanti    line : 

your shoes are great but get those away from me dankrand
actor - jakarta joshua pandelaki
acceptance is power    traveling culinary  enthusiast   jennarnelita
#11 ano
  manajemen   fakultas ekonomi dan bisnis   universitas pendidikan ganesha   praga
sampoerna academy asbi  widyaasthari
penghageng at tepas tandhayekti @kratonjogja it project manager aspiring entrepreneur  wants to be more ladylike but hopelessly loves all things geeky  gkr hayu
selamat datang di akun twitter resmi dari wuling motors  wuling motors - drive for a better life wulingmotorsid
interior designer adhityadesigner
i am limited edition there is only one me  pingky martha margaretha
9 5 bisma karisma
mc komedi  cp : 081333412728 insan nur akbar
#9anteng #solvieron   liverpool uqjhs smauq   yang hebat bukan yg menang tapi yg menang adalah yg hebat   089614477624   ayeeeee            muhammad faishal
beritasatu media holdings: https:  t co oea5bcf5oj dan beritasatu tv  beritasatu
the girl who make u curiousðÿœ¬ â€¢ i

 gusti rayhan gibayus
a savory cuisine of timeless tradition sate khas senayan
mom wife bo$$ cinta ramlan nunes
ori steak   pasta radio dalem no 47a lian firman
hyper-scaling @ristekcsui @ristekoss   deadlining assignments @univ indonesia   most outstanding student @fasilkom ui   prev  @ycombinator @pintuid   #bitcoin jonathan filbert
orange is the colour on me   cat is my therapy ðÿˆ   # diskusidinda   tukang kain di @ watuku co   orangelava
 faisal
akun resmi badan eksekutif mahasiswa fakultas ilmu pengetahuan budaya universitas indonesia 2021 bem fib ui
mechanical engineering hmm ft  upri mks djoe
to live `
 ë¹µë§ˆë‹´
cp : iman 0811145214 warman 08158916561 imam darto project
official twitter account of kurniaven setiawan    a true gentleman never leaves his lady- @delpieroale    juventino â˜…â˜…â˜…    tukang ukur jalanan ibukota~    enter wind~
cs ui '14 irinbn
apabila anda seorang muslim wni berusia 20-35 tahun dan pernah mengonsumsi makanan korea dimohon kesediaan dan waktunya u

fakultas hukum universitas indonesia 2013   next law enforcer  muhammad fadhil
s3 monash university australia pengalaman lebih dari 30 tahun di pemerintahan bumn dan pemerhati pariwisata internasional dan pelindung umkm indonesia  i ketut mardjana
 js
28 dec 2003   komunitas olahraga anak muda yang memiliki daya cipta tinggi   ingin menjadikan indonesia yang terbaik  http:  t co zd4jkuho ball star indonesia
1st  nazwa
smkn1 tg selor fkip akuntansi-univ sanata dharma  usd  yogyakarta  kiki ping
singer christ follower â¤ï¸ grateful wife   mom singing jobsâž¡08111830269  sms whatsapp  alenawu
akun resmi polsek cibatu purwakarta telp:  0264  202241 #polsekcibatupurwakarta polsek cibatu purwakarta
gg  selot anggi
tv host â€¢ mc â€¢ life enthusiast â€¢ business contact : deya +62 812 9607073   ruly +62 811 9851699 olivia fendry
i am my own woman - [ instagram: thickaperempuan ] [ mustikayuliandri@gmail com ] mustika t  yuliandri
olahraga klub politik   pemerintahan hiburan musik ekonomi   

jurnalisme data  media analisis  infografik ciamik  tirtoid
ð’œð’†ð’†ð’‘ ð’„ð’‚ð’ð’ž ð’‚ð’ð’… ð’ƒð’†ð’ð’šð’†ð’—ð’† ð’•ð’ ð‘®ð’ð’… ðÿ˜‡ ðÿœˆ


In [211]:
collection.head()

,username,name,description
0,@BobidDams,,industrial engineering of pancasila university...
1,@GrandIndo,,official account of grand indonesia a one-stop...
2,@persisofficial,,official account of persis solo #laskarsamber...
3,@hamsil_,,i am you hammam silmi
4,@chrisryantonius,,chris ryantonius


In [212]:
collection.to_csv("input.csv".format(2), index=False)

In [213]:
#membuat method untuk menghitung berapa banyak kecocokan kata pada description dan name yang sesuai dengan leksikon yang dibuat

def ekstraksi_sentimen(list_tweet):
    pendidikan_penelitian = ["undergraduate", "pendidikan", "mahasiswa", "mahasiswi", "departemen", "bem", "kampus", "fakultas", "faculty", "universitas", "institut", "ui", "ugm", "itb", "unj", "uin", "stan", "student", "badan",  "eksekutif", "university", "academy", "researcher", "engineering", "penelitian", "sma", "sman", "perguruan", "phd", "riset", "studi", "academy", "edukasi", "institute", "institution", "lecturer", "research", "school", "pelajar", "akademi", "politeknik", "unpad", "dekan", "dosen", "teacher", "teach", "educator", "learning", "guru", "diploma", "himpunan", "Kemahasiswaan", "smpn", "sd", "sdn"]
    sains_teknologi = ["programmer", "fullstack", "developer", "develop", "teknologi", "tech", "technology" "gadget", "smartphone", "software", "engineer", "teknik", "informatika", "analyst", "sains", "science", "biologi", "android", "spring", "astronomi",  "ilmu", "pengetahuan", "engineering", "product", "programming", "startup", "android", "computer", "klimatologi", "meteorologi", "mobile", "network","sciences", "statistik", "system", "physician", "pemrograman", "full-stack", "backend", "back-end", "frontend", "front-end", "telekomunikasi", "google", "googledevexpert", "internet", "it", "coding", "data"]
    kesehatan = ["dokter", "kesehatan", "doctor", "klinik", "gejala", "health", "bpjs", "apoteker", "sehat", "clinical", "medical", "medicine", "medicines", "vaccines", "nursing", "obgyn", "obat", "obgynee", "rumah", "sakit", "surgeon", "urologist", "covid", "hospital", "internist", "rsud"]
    ekonomi_bisnis = ["pt", "diskon", "bank", "bisnis", "ekonomi", "bitcoin", "wirausaha", "entrepreneur", "call", "center", "konsultan", "perbelanjaan", "market", "trader", "trading", "financial", "founder", "investor", "keuangan", "motor", "perekonomian", "delivery", "saham", "harga", "company", "niaga", "economics", "operator", "otoritas", "pajak", "persero", "mall", "accountant", "shopping", "business", "cakepreuner", "toko", "ojk", "commerce", "promosi", "promo", "promotion", "promotions", "investment", "gojek", "ihsg", "pelanggan", "restaurant", "restoran", "customer", "penerbit", "giveaway"]
    sosial_kemasyarakatan = ["sosial", "kementerian" , "menteri", "dpr", "hukum", "masyarakat", "polsek", "panglima", "tni", "pmi", "kedubes", "humas", "aduan", "advocate", "advokasi", "anggota", "dewan", "partai", "badan", "provinsi", "biro", "organisasi", "hutan", "call", "center", "hak", "ham", "human", "pemda", "pemerintah", "polda", "gerakan", "perwakilan", "protokol", "kabinet", "pelayanan", "komisi", "lingkungan", "rakyat", "tv", "ministry", "gerakan", "kedinasan", "law", "politician", "political", "politics", "presiden", "gubernur", "walikota", "yayasan", "asasi", "bupati", "direktorat","kebijakan", "layanan", "lembaga", "pemprov", "pimpinan", "politik", "nonprofit", "pkb", "pks", "pelayan", "pksejahtera", "policy", "polisi", "balai", "kepala", "ketua", "activist", "kepolisian", "konseling", "kpk", "mpr", "ketum", "ulama", "dprd", "fraksi", "non-profit", "kepolisian","coordinating", "polres", "parliament", "pemasyarakatan", "pembina", "government", "governor", "dapil", "motivator", "jenderal"]
    media = ["media", "news", "presenter", "berita", "informasi", "jurnalis", "journalist", "berbagi", "breaking", "broadcaster", "info", "journalism", "jurnalisme", "kabar", "culture", "publikasi", "radio", "post", "share", "information", "network", "reports", "newspaper", "portal", "report", "speaker", "redaksi", "koran", "majalah", "tabloid", "magazine"]
    hospitality_tourism = ["wisata", "kuliner", "tourism", "pariwisata", "monumen", "museum", "safari", "perhotelan", "hotel", "taman", "nasional", "tour", "tourist", "cafe", "conservation", "recreational", "resort", "plaza", "theme", "api", "zoo"]
    olahraga = ["futsal", "persib", "athlete", "olympian", "bola", "sepeda",  "football", "cup", "bola", "basketball", "timnas", "volleyball", "olahraga", " fc", "esports", "esport", "taekwondo", "persipura", "sport", "sports", "atlet", "player", "badminton", "bulutangkis", "champions"]
    hiburan = ["cp", "contact", "ig", "instagram", "podcast", "music", "musik", "jazz", "album", "sing", "singer", "social", "vlogger", "meme", "actor", "model", "actress", "acting", "actress singer", "rapper", "festival", "booking", "game", "gaming", "gamers", "gamer", "director", "blogger", "lucu", "creativepreneur", "comedian", "influencer", "komika", "musical", "artist", "performing", "performance", "komedi", "producer", "standup", "cp:", "stand", "up", "comedy", "artists", "arts", "academy", "lifestyle", "talk", "talkshow", "tiktok", "pertunjukan", "mc", "actris", "host", "sportcaster", "shitposting", "vokal", "vocal", "album", "acting", "akting", "jkt48", "musician", "single", "content", "youtube", "channel", "tv", "host", "mc", "horror", "band", "vocalist", "vocalista", "guitar"]
    seni = ["writer", "puisi", "poetry", "penulis", "buku", "author", "single", "buku", "teater",  "novelis", "illustrator", "sastra", "seni", "photographer", "fotografer","comic", "comics", "cartoonist", "menulis", "write", "illustrations", "dancer", "art", "draw", "gramedia", "kebahasaan", "kesastraan", "fashion", "boutique", "design", "film", "filmmaker", "teater", "ballet"]
    hobi = ["fanbase", "fan", "fans", "friendbase", "komunitas", "community", "berita", "berita", "bola", "football", "gowes", "sepeda", "bts", "army", "juventus", "pecinta", "sepakbola", "sepeda", "manchester", "city", "games", "game", "page", "penggiat", "peminat", "anime", "super", "juniors", "junior", "resep", "supporter", "support", "club", "volleyball", "fansbase", "fanpage", "fans", "fanspage", "menfess", "milan", "traveling", "football", "asosiasi", "nct", "taekwondo", "blink", "mamamoo"]
    fitur_sentimen_all = []
    for tweet in list_tweet:
        # inisiasi value
        emosi = ["pendidikan_penelitian", "sains_teknologi", "kesehatan", "ekonomi_bisnis", "sosial_kemasyarakatan", 
                 "media", "hospitality_tourism", "olahraga", "hiburan", "seni", "hobi"]
        value = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        emosi_value = {}
        for i in range(len(emosi)):
            emosi_value[emosi[i]] = value[i]
        list_kata = tweet.split()
        for k in list_kata:
            if k in pendidikan_penelitian:
                emosi_value["pendidikan_penelitian"] += 1
            if k in sains_teknologi:
                emosi_value["sains_teknologi"] += 1
            if k in kesehatan:
                emosi_value["kesehatan"] += 1
            if k in ekonomi_bisnis:
                emosi_value["ekonomi_bisnis"] += 1
            if k in sosial_kemasyarakatan:
                emosi_value["sosial_kemasyarakatan"] += 1
            if k in media:
                emosi_value["media"] += 1
            if k in hospitality_tourism:
                emosi_value["hospitality_tourism"] += 1
            if k in olahraga:
                emosi_value["olahraga"] += 1
            if k in hiburan:
                emosi_value["hiburan"] += 1
            if k in seni:
                emosi_value["seni"] += 1 
            if k in hobi:
                emosi_value["hobi"] += 1 
        fitur_sentimen_perkalimat = list(emosi_value.values())
        fitur_sentimen_all.append(fitur_sentimen_perkalimat)
        print(fitur_sentimen_all)
        max_value = max(fitur_sentimen_all[0])
        if(max_value != 0):
            result = fitur_sentimen_all[0].index(max(fitur_sentimen_all[0]))
        else:
            result = -1
    return result

In [214]:
nf = pd.read_csv("input.csv")

In [215]:
nf.head()

,username,name,description
0,@BobidDams,NaN,industrial engineering of pancasila university...
1,@GrandIndo,NaN,official account of grand indonesia a one-stop...
2,@persisofficial,NaN,official account of persis solo #laskarsamber...
3,@hamsil_,NaN,i am you hammam silmi
4,@chrisryantonius,NaN,chris ryantonius


In [216]:
nf['description'] = nf["description"].fillna(" ")

In [217]:
#menentukan prediksi setiap akun berdasarkan nilai terbesar pada array yang dibuat

for i in range(0,4153):
    description = nf.loc[i, 'description']
    result = ekstraksi_sentimen([description])
    if(result == 0):
        text_result = "pendidikan dan penelitian"
    elif(result == 1):
        text_result = "sains dan teknologi"
    elif(result == 2):
        text_result = "kesehatan"
    elif(result == 3):
        text_result = "ekonomi dan bisnis"
    elif(result == 4):
        text_result = "sosial kemasyarakatan"
    elif(result == 5):
        text_result = "media"
    elif(result == 6):
        text_result = "hospitality & tourism"
    elif(result == 7):
        text_result = "olahraga"
    elif(result == 8):
        text_result = "hiburan"
    elif(result == 9):
        text_result = "seni"
    elif(result == 10):
        text_result = "hobi"

    dfresult['0'] = i
    dfresult['1'] = text_result
    nc.loc[i] = pd.Series(dfresult)

[[3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0]]
[[0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0]]
[[4, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]]
[[1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0,

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 6, 2, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 1, 0, 1, 0, 0,

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 4]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0]]
[[0, 1, 0, 1, 2, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]
[[0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]]
[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0,

[[0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 1]]
[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0]]
[[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]
[[0, 0, 0, 0, 1, 4, 0, 0, 0, 1, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]]
[[3, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 1]]
[[0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 2, 0, 2, 0,

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]]
[[0, 0, 0, 2, 2, 0, 0, 0, 2, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[5, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
[[0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 1,

...

[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]
[[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 4, 0, 0, 1, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]
[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1]]
[[0, 1, 0, 2, 0, 1, 0, 2, 1, 0, 1]]
[[0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]
[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0,

[[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]
[[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]
[[2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 3, 0, 0, 0, 0, 1, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0,

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 2, 2, 0, 0, 0, 1, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 0]]
[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0,

[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]]
[[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[3, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]]
[[0, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0]]
[[2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
[[1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]]
[[3, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]]
[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2]]
[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
[[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0,

[[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 1, 0, 0, 2, 1, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 2,

In [218]:
nc.head(10)

,0,1
0,0,pendidikan dan penelitian
1,1,ekonomi dan bisnis
2,2,ekonomi dan bisnis
3,3,ekonomi dan bisnis
4,4,ekonomi dan bisnis
5,5,hiburan
6,6,hiburan
7,7,pendidikan dan penelitian
8,8,pendidikan dan penelitian
9,9,pendidikan dan penelitian


In [32]:
nc.to_csv("submission10.csv".format(2), index=False)